In [8]:
from logging import raiseExceptions
from tokenize import Double
import networkx as nx
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
from vec3 import Vec3
import meshplot as mp
import torch
torch.manual_seed(125)
import random
random.seed(125)
import torch_f as torch_f
from modelo import Node, GRASSEncoder, GRASSDecoder
import wandb


In [9]:
use_gpu = True
device = torch.device("cuda:0" if use_gpu and torch.cuda.is_available() else "cpu")

def count_fn(f):
    def wrapper(*args, **kwargs):
        wrapper.count += 1
        return f(*args, **kwargs)
    wrapper.count = 0
    return wrapper

In [10]:
def plotTree( root, dec ):
    graph = nx.Graph()
    root.toGraph( graph, 0, dec)
    edges=nx.get_edge_attributes(graph,'procesada')

    p = mp.plot( np.array([ graph.nodes[v]['posicion'] for v in graph.nodes]), shading={'point_size':0.1}, return_plot=True)

    for arista in graph.edges:
        p.add_lines( graph.nodes[arista[0]]['posicion'], graph.nodes[arista[1]]['posicion'])

    return 

def subplotTree( root, dec, d, c = "black"):
    graph = nx.Graph()
    root.toGraph( graph, 0, dec)
    edges=nx.get_edge_attributes(graph,'procesada')

    shading={'point_size':0.1, "point_color": c, "line_color": c}
    #d = mp.plot( np.array([ graph.nodes[v]['posicion'] for v in graph.nodes]), shading={'point_size':0.1}, return_plot=True)
    d.add_points( np.array([ graph.nodes[v]['posicion'] for v in graph.nodes]), shading = shading)
    for arista in graph.edges:
        d.add_lines( graph.nodes[arista[0]]['posicion'], graph.nodes[arista[1]]['posicion'], shading  = shading)


    return d

def traverse(root, tree):
       
        if root is not None:
            traverse(root.left, tree)
            tree.append((root.radius, root.data))
            traverse(root.right, tree)
            return tree

def traversem(root):
       
        if root is not None:
            traversem(root.left)
            print(root)
            traversem(root.right)
            return 

def traverse_2(tree1, tree2, t_l):
       
        if tree1 is not None:
            traverse_2(tree1.left, tree2.left, t_l)
            if tree2:
                t_l.append((tree1.radius, tree2.radius))
                print((tree1.radius, tree2.radius))
            else:
                t_l.append(tree1.radius)
                print((tree1.radius))
            traverse_2(tree1.right, tree2, t_l)
            return t_l
            

def traverse_conexiones(root, tree):
        """
        traverse function will print all the node in the tree.
        """
        if root is not None:
            traverse_conexiones(root.left, tree)
            if root.right is not None:
                tree.append((root.data, root.right.data))
            if root.left is not None:
                tree.append((root.data, root.left.data))
            traverse_conexiones(root.right, tree)
            return tree

def createNode(data, radius, position = None, left = None, right = None, cl_prob = None, ce = None, mse=None):
        """
        Utility function to create a node.
        """
        return Node(data, radius, position, left, right, cl_prob, ce, mse)
 
def deserialize(data):
    if  not data:
        return 
    nodes = data.split(';')  
    #print("node",nodes[3])
    def post_order(nodes):
                
        if nodes[-1] == '#':
            nodes.pop()
            return None
        node = nodes.pop().split('_')
        data = int(node[0])
        #radius = float(node[1])
        #print("node", node)
        #breakpoint()
        radius = node[1]
        #print("radius", radius)
        rad = radius.split(",")
        rad [0] = rad[0].replace('[','')
        rad [3] = rad[3].replace(']','')
        r = []
        for value in rad:
            r.append(float(value))
        #r =[float(num) for num in radius if num.isdigit()]
        r = torch.tensor(r, device=device)
        #breakpoint()
        root = createNode(data, r)
        root.right = post_order(nodes)
        root.left = post_order(nodes)
        
        return root    
    return post_order(nodes)    


def read_tree(filename, dir):
    #with open('./prof6/' +filename, "r") as f:
    #with open('./trees/' +filename, "r") as f:
    with open('./' +dir +'/' +filename, "r") as f:
        byte = f.read() 
        return byte

def numerar_nodos(root, count):
    if root is not None:
        numerar_nodos(root.left, count)
        root.data = len(count)
        count.append(1)
        numerar_nodos(root.right, count)
        return 


def traversefeatures(root, features):
       
    if root is not None:
        traversefeatures(root.left, features)
        features.append(root.radius)
        traversefeatures(root.right, features)
        return features

def norm(root, minx, miny, minz, minr, maxx, maxy, maxz, maxr):
    
    if root is not None:
        mx = minx.clone().detach()
        my = miny.clone().detach()
        mz = minz.clone().detach()
        mr = minr.clone().detach()
        Mx = maxx.clone().detach()
        My = maxy.clone().detach()
        Mz = maxz.clone().detach()
        Mr = maxr.clone().detach()
       
        root.radius[0] = (root.radius[0] - minx)/(maxx - minx)
        root.radius[1] = (root.radius[1] - miny)/(maxy - miny)
        root.radius[2] = (root.radius[2] - minz)/(maxz - minz)
        root.radius[3] = (root.radius[3] - minr)/(maxr - minr)
        
        norm(root.left, mx, my, mz, mr, Mx, My, Mz, Mr)
        norm(root.right, mx, my, mz, mr, Mx, My, Mz, Mr)
        return 

def normalize_features(root):
    features = []
    features = traversefeatures(root, features)
    
    x = [tensor[0] for tensor in features]
    y = [tensor[1] for tensor in features]
    z = [tensor[2] for tensor in features]
    r = [tensor[3] for tensor in features]
 
    norm(root, min(x), min(y), min(z), min(r), max(x), max(y), max(z), max(r))

    return 

def traversefeatures(root, features):
       
    if root is not None:
        traversefeatures(root.left, features)
        features.append(root.radius)
        traversefeatures(root.right, features)
        return features

def calcularLossAtributo(nodo, radio):
    if nodo is None:
        return
    radio = radio.reshape(-1,4)
    nodo = nodo.radius.reshape(-1,4)
    l2    = nn.MSELoss()
   
    mse = l2(radio, nodo)
    return mse

def searchNode(node, key):
     
    if (node == None):
        return False
 
    if (node.data == key):
        return node
        
 
    """ then recur on left subtree """
    res1 = searchNode(node.left, key)
    # node found, no need to look further
    if res1:
        return res1
 
    """ node is not found in left,
    so recur on right subtree """
    res2 = searchNode(node.right, key)
    return res2

def getLevelUtil(node, data, level):
    if (node == None):
        return 0
 
    if (node.data == data):
        return level
 
    downlevel = getLevelUtil(node.left, data, level + 1)

    if (downlevel != 0):
        return downlevel
 
    downlevel = getLevelUtil(node.right, data, level + 1)
    return downlevel
 
# Returns level of given data value
 
 
def getLevel(node, data):
    return getLevelUtil(node, data, 1)
 

def set_tree_level(data_loader):
    for d in data_loader:
        for data in d:
            count = []
            numerar_nodos(data, count)
            c = []
            n_nodes = data.count_nodes(data, c)
            for x in range(0, len(n_nodes)):
                level = getLevel(data, x)
                if (level):
                    node = searchNode(data, x)
                    node.level = getLevel(data, x)
                else:
                    print(x, "is not present in tree")
            tree_level = []
            data.get_tree_level(data, tree_level)
            data.set_tree_level(data, sum(tree_level))

def calcularLossEstructura(cl_p, original, mult):
        
        if original is None:
            return
        ce = nn.CrossEntropyLoss(weight = mult)

        if original.childs() == 0:
            vector = [1, 0, 0] 
        if original.childs() == 1:
            vector = [0, 1, 0]
        if original.childs() == 2:
            vector = [0, 0, 1] 


        c = ce(cl_p, torch.tensor(vector, device=device, dtype = torch.float).reshape(1, 3))
        return c

def number_nodes(data_loader, batch_size):
    n_no = []
    qzero = 0
    qOne = 0
    qtwo = 0
    for batch in data_loader:
        for tree in batch:
            count = []
            n = tree.count_nodes(tree, count)
            n_no.append(len(n))
            li = []
            tree.traverseInorderChilds(tree, li)
            zero = [a for a in li if a == 0]
            one = [a for a in li if a == 1]
            two = [a for a in li if a == 2]
            qzero += len(zero)
            qOne += len(one)
            qtwo += len(two)

    nprom = np.mean(n_no)
    qzero /= len(data_loader)*batch_size
    qOne /= len(data_loader)*batch_size
    qtwo /= len(data_loader)*batch_size
    if round(qzero) == 0:
        qzero = 1
    if round(qOne) == 0:
        qOne = 1
    if round(qtwo) == 0:
        qtwo = 1
    mult = torch.tensor([1/round(qzero),1/round(qOne),1/round(qtwo)], device = device)
    return mult
    

In [11]:
def my_collate(batch):
    return batch


class tDataset(Dataset):
    def __init__(self, l, dir, transform=None):
        self.names = l
        self.transform = transform
        self.data = [] #lista con las strings de todos los arboles
        for file in self.names:
            self.data.append(read_tree(file, dir))
        self.trees = []
        for tree in self.data:
            deserial = deserialize(tree)
            normalize_features(deserial)
            self.trees.append(deserial)

    def __len__(self):
        return len(self.names)

    def __getitem__(self, idx):
        #file = self.names[idx]
        #string = read_tree(file)
        tree = self.trees[idx]
        return tree

batch_size = 1

In [12]:
import uuid
from ipywidgets import Output, HBox
rendertype = 'JUPYTER'

class Subplot():
    def __init__(self, data, view, s):
        if data == None:
            self.rows = []
            self.hboxes = []
        else:
            self.rows = data.rows
        if s[0] != 1 or s[1] != 1:
            if data == None: # Intialize subplot array
                cnt = 0
                for r in range(s[0]):
                    row = []
                    for c in range(s[1]):
                        row.append(Output())
                        cnt += 1
                    self.rows.append(row)

                for r in self.rows:
                    hbox = HBox(r)
                    if rendertype == "JUPYTER":
                        display(hbox)
                    self.hboxes.append(hbox)
        
            out = self.rows[int(s[2]/s[1])][s[2]%s[1]]
            if rendertype == "JUPYTER":
                with out:
                    display(view._renderer)
            self.rows[int(s[2]/s[1])][s[2]%s[1]] = view

    def save(self, filename=""):
        if filename == "":
            uid = str(uuid.uuid4()) + ".html"
        else:
            filename = filename.replace(".html", "")
            uid = filename + '.html'

        s = ""
        imports = True
        for r in self.rows:
            for v in r:
                s1 = v.to_html(imports=imports, html_frame=False)
                s = s + s1
                imports = False

        s = "<html>\n<body>\n" + s + "\n</body>\n</html>"
        with open(uid, "w") as f:
            f.write(s)
        print("Plot saved to file %s."%uid)

    def to_html(self, imports=True, html_frame=True):
        s = ""
        for r in self.rows:
            for v in r:
                s1 = v.to_html(imports=imports, html_frame=html_frame)
                s = s + s1
                imports = False

        return s

In [13]:
def subplot(f, c = 'red', uv=None, n=None, shading={}, s=[1, 1, 0], data=None, **kwargs):
    
    shading={'point_size':0.1, "point_color": c, "line_color": c, "width":400, "height":400}
    view = mp.Viewer(settings = {"width": 500, "height": 500, "antialias": True, "scale": 1.5, "background": "#ffffff",
                "fov": 30})

    obj = view.add_points(np.array([ f.nodes[v]['posicion'] for v in f.nodes]), shading=shading)
    #view.add_lines(v, f, shading=shading)
    for arista in f.edges:
        obj = view.add_lines( f.nodes[arista[0]]['posicion'], f.nodes[arista[1]]['posicion'], shading  = shading)
   
    subplot = Subplot(data, view, s)
    #subplot = view
    return subplot

def sTree( root, dec, s, c, d=None):
    graph = nx.Graph()
    root.toGraph( graph, 0, dec)
    #edges=nx.get_edge_attributes(graph,'procesada')

    #shading={'point_size':0.1, "point_color": c, "line_color": c}
    #d = subplot( graph, c, s)
    if d:
        subplot(graph, c=c, s=s, data = d)
    else:
        
        d = subplot(graph, c=c, s=s)
    
    #for arista in graph.edges:
    #    d.add_lines( graph.nodes[arista[0]]['posicion'], graph.nodes[arista[1]]['posicion'], shading  = shading)
    return d

In [14]:
folder = "dataset"
#t_list = os.listdir("./Trees/")
t_list = ['ArteryObjAN201-15_tree.dat', 'ArteryObjAN168-10_tree.dat', 'ArteryObjAN161-14_tree.dat', 'ArteryObjAN186-12_tree.dat', 'ArteryObjAN1-2_tree.dat', 'ArteryObjAN178-5_tree.dat', 'ArteryObjAN116-11_tree.dat', 'ArteryObjAN215-11_tree.dat', 'ArteryObjAN215-7_tree.dat', 'ArteryObjAN135-18_tree.dat', 'ArteryObjAN191-10_tree.dat', 'ArteryObjAN204-18_tree.dat', 'ArteryObjAN215-8_tree.dat', 'ArteryObjAN208-17_tree.dat', 'ArteryObjAN11-4_tree.dat', 'ArteryObjAN119-19_tree.dat', 'ArteryObjAN212-9_tree.dat', 'ArteryObjAN197-1_tree.dat', 'ArteryObjAN44-10_tree.dat', 'ArteryObjAN163-4_tree.dat']
t_list = os.listdir("./p10/")
t_list = ['553.dat', '727.dat', '666.dat', '513.dat', '901.dat', '410.dat', '986.dat', '651.dat', '364.dat', '917.dat', '919.dat', '250.dat', '467.dat', '786.dat', '152.dat', '39.dat', '402.dat', '338.dat', '254.dat', '449.dat', '1040.dat', '429.dat', '137.dat', '540.dat', '760.dat', '515.dat', '968.dat', '597.dat', '682.dat', '1042.dat', '1050.dat', '68.dat', '977.dat', '802.dat', '1037.dat', '422.dat', '186.dat', '29.dat', '183.dat', '999.dat', '840.dat', '645.dat', '800.dat', '99.dat', '107.dat', '441.dat', '696.dat', '543.dat', '925.dat', '899.dat', '698.dat', '269.dat', '348.dat', '577.dat', '618.dat', '370.dat', '720.dat', '752.dat', '424.dat', '516.dat', '590.dat', '530.dat', '544.dat', '748.dat', '785.dat', '769.dat', '532.dat', '534.dat', '360.dat', '382.dat', '576.dat', '589.dat', '823.dat', '648.dat', '721.dat', '167.dat', '833.dat', '263.dat', '358.dat', '310.dat', '817.dat', '882.dat', '904.dat', '409.dat', '948.dat', '87.dat', '804.dat', '571.dat', '951.dat', '72.dat', '966.dat', '367.dat', '110.dat', '549.dat', '473.dat', '936.dat', '708.dat', '118.dat', '216.dat', '661.dat']
dataset = tDataset(t_list, 'p10')
data_loader = DataLoader(dataset, batch_size = batch_size, shuffle=True, collate_fn=my_collate)
    
i = 0
d = None
for batch in data_loader:
    for input in batch:
        if d is None:
            d = sTree(input, False, c = "black", s = [50,2,i])
        else:
            sTree(input, False, c = "black", s = [50,2,i], d=d)

        #sTree(input, True,  s = [1,1,i+1], c = "black", d=d)
        i += 1
d.save("datasetplot.html")

t_list 100


Plot saved to file datasetplot.html.
